In [1]:
from datasets import load_dataset
import json

from openai_service import get_chat_completion
from prompts import *

/Users/abhishekanand/miniforge3/envs/focus-llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("allenai/ai2_arc", "ARC-Challenge")
df = dataset['train'].to_pandas()

df.head()

,id,question,choices,answerKey
0,Mercury_SC_415702,George wants to warm his hands quickly by rubb...,"{'text': ['dry palms', 'wet palms', 'palms cov...",A
1,MCAS_2009_5_6516,Which of the following statements best explain...,"{'text': ['The refrigerator door is smooth.', ...",B
2,Mercury_7233695,A fold observed in layers of sedimentary rock ...,"{'text': ['cooling of flowing magma.', 'conver...",B
3,Mercury_7041615,Which of these do scientists offer as the most...,"{'text': ['worldwide disease', 'global mountai...",D
4,Mercury_7041860,A boat is acted on by a river current flowing ...,"{'text': ['west', 'east', 'north', 'south'], '...",B


In [3]:
def build_prompt_for_sample(id, df):
    question = df.iloc[id].question
    choices = df.iloc[id].choices
    options = str(choices['text'].tolist())
    return concept_guided_reasoning_prompt.replace("{question}", question).replace("{options}", options)

In [4]:
print(get_chat_completion(build_prompt_for_sample(0, df)))

('{\n  "concepts": [\n    {\n      "name": "skin surface",\n      "description": "outer layer of the skin on the palms"\n    },\n    {\n      "name": "dry palms",\n      "description": "palms without moisture or any substances on them"\n    },\n    {\n      "name": "wet palms",\n      "description": "palms covered with water"\n    },\n    {\n      "name": "palms covered with oil",\n      "description": "palms coated with oil"\n    },\n    {\n      "name": "palms covered with lotion",\n      "description": "palms coated with lotion"\n    },\n    {\n      "name": "heat production",\n      "description": "generation of warmth on the skin by rubbing hands"\n    }\n  ],\n  "relationships": [\n    {\n      "concept": "skin surface",\n      "relationship": "can be influenced by",\n      "related_concept": "dry palms",\n      "description": "Dry palms are a state of the skin surface."\n    },\n    {\n      "concept": "skin surface",\n      "relationship": "can be influenced by",\n      "relate

In [6]:
print(build_prompt_for_sample(0, df))

Question: - George wants to warm his hands quickly by rubbing them. Which skin surface will produce the most heat?
Options: - ['dry palms', 'wet palms', 'palms covered with oil', 'palms covered with lotion']

All the main real world concepts essential to answer the question are: 
List them down along with one line description
Remember to return the response in a json structure as defined below - 

"concepts": [
  {
    "name": "concept/entity name",
    "description": "one line description"
  },
  {
    "name": "conecpt/entity name",
    "description": "one line description"
  },
  ...
]

[your concepts list]

In manner of ontology, describe the relationship for each concept to the other concepts in the context of the sentence in a structured manner along with one line description for each are:
Remember to return the response in a json structure as defined below -

"relationships": [
	{
		"concept": "concept1",
		"relationship": "relation",
		"related_concept": "concept2",
		"descripti